In [533]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OrdinalEncoder

In [534]:
X = datasets.fetch_covtype().data[:5000]
y = datasets.fetch_covtype().target[:5000]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

print(X_train.shape, y_train.shape)

enc = OrdinalEncoder()
y_train = enc.fit_transform(y_train.reshape(-1, 1)).reshape(-1, )
y_test = enc.transform(y_test.reshape(-1, 1)).reshape(-1, )
print(np.unique(y_train))  # 7分类任务

X_train, X_test, y_train, y_test = tf.constant(X_train), tf.constant(X_test), tf.constant(y_train), tf.constant(y_test)

(4500, 54) (4500,)
[0. 1. 2. 3. 4. 5. 6.]


In [535]:
inputs = tf.keras.Input(shape=(54,))
x = tf.keras.layers.Dense(256, activation='tanh')(inputs)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dense(128)(x)
predictions = tf.keras.layers.Dense(7)(x)
model = tf.keras.Model(inputs=inputs, outputs=predictions)

model.summary()

Model: "model_81"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_84 (InputLayer)       [(None, 54)]              0         
                                                                 
 dense_273 (Dense)           (None, 256)               14080     
                                                                 
 dropout_27 (Dropout)        (None, 256)               0         
                                                                 
 dense_274 (Dense)           (None, 128)               32896     
                                                                 
 dense_275 (Dense)           (None, 128)               16512     
                                                                 
 dense_276 (Dense)           (None, 7)                 903       
                                                                 
Total params: 64,391
Trainable params: 64,391
Non-trainabl

In [536]:
# Configures the model for training.
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # 优化器
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  # 损失函数
              metrics=['accuracy'])  # 评估函数

In [537]:
# Trains the model for a fixed number of epochs (iterations on a dataset).
model.fit(X_train,  # Input data
          y_train,  # Target data
          # x是生成器或 tf.data.Dataset 的对象时,将忽略此参数
          # Boolean (whether to shuffle the training data before each epoch) or str (for 'batch'). This argument is ignored when x is a generator or an object of tf.data.
          shuffle=True,
          # Integer or None. Number of samples per gradient update. If unspecified, batch_size will default to 32.
          batch_size=64,
          epochs=10,  # Integer. Number of epochs to train the model.
          # 'auto', 0, 1, or 2. Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch. 'auto' defaults to 1 for most cases
          verbose=2,
          # Data on which to evaluate the loss and any model metrics at the end of each epoch. The model will not be trained on this data.
          validation_data=(X_test, y_test),
          # Float between 0 and 1. Fraction of the training data to be used as validation data. The model will set apart this fraction of the training data, will not train on it, and will evaluate the loss and any model metrics on this data at the end of each epoch.
          # validation_split=0.2 # 验证数据集占训练数据集的比例,取值范围为0~1
          )

Epoch 1/10
71/71 - 1s - loss: 1.4301 - accuracy: 0.4562 - val_loss: 1.1324 - val_accuracy: 0.5380 - 536ms/epoch - 8ms/step
Epoch 2/10
71/71 - 0s - loss: 1.2301 - accuracy: 0.5084 - val_loss: 1.0904 - val_accuracy: 0.5760 - 215ms/epoch - 3ms/step
Epoch 3/10
71/71 - 0s - loss: 1.2015 - accuracy: 0.5176 - val_loss: 1.0643 - val_accuracy: 0.5720 - 206ms/epoch - 3ms/step
Epoch 4/10
71/71 - 0s - loss: 1.1602 - accuracy: 0.5242 - val_loss: 1.0790 - val_accuracy: 0.5500 - 224ms/epoch - 3ms/step
Epoch 5/10
71/71 - 0s - loss: 1.1495 - accuracy: 0.5287 - val_loss: 1.0505 - val_accuracy: 0.5680 - 201ms/epoch - 3ms/step
Epoch 6/10
71/71 - 0s - loss: 1.1336 - accuracy: 0.5329 - val_loss: 1.0041 - val_accuracy: 0.5800 - 177ms/epoch - 2ms/step
Epoch 7/10
71/71 - 0s - loss: 1.1211 - accuracy: 0.5436 - val_loss: 0.9953 - val_accuracy: 0.6040 - 189ms/epoch - 3ms/step
Epoch 8/10
71/71 - 0s - loss: 1.1109 - accuracy: 0.5482 - val_loss: 0.9257 - val_accuracy: 0.6160 - 209ms/epoch - 3ms/step
Epoch 9/10
71/71

In [538]:
# Returns the loss value & metrics values for the model in test mode.
'''
batch_size:Integer or None. Number of samples per batch of computation. If unspecified, batch_size will default to 32. Do not specify the batch_size if your data is in the form of a dataset, generators, or keras.utils.Sequence instances (since they generate batches).
verbose:0 or 1. Verbosity mode. 0 = silent, 1 = progress bar.
'''
loss, metric = model.evaluate(X_test, y_test, batch_size=32, verbose=0)

print("test loss:", loss)
print("test acc:", metric)

test loss: 0.9256449341773987
test acc: 0.6340000033378601


In [539]:
# Generates output predictions for the input samples.
'''
batch_size:Integer or None. Number of samples per batch. If unspecified, batch_size will default to 32. Do not specify the batch_size if your data is in the form of dataset, generators, or keras.utils.Sequence instances (since they generate batches).
verbose:Verbosity mode, 0 or 1.
'''
predictions = model.predict(X_test[:10],  # 预测10条样本
                            batch_size=32, verbose=0)
print('predictions shape:', predictions)

predictions shape: [[-2.861379   -0.84848404  2.4295695   3.3494418   1.1932477   1.0286329
  -6.627928  ]
 [-1.6492828   0.33578384  2.1444478   0.30763477  0.11676084  1.9300216
  -3.9182441 ]
 [ 0.862491    2.995561   -0.3893175  -2.5053957   3.033133    0.41988632
  -3.0754466 ]
 [ 0.03761072  1.6790218  -1.1376354   1.8506185   0.19919974 -1.1811516
  -3.1601036 ]
 [ 0.18730937  2.4615493  -0.07888934 -1.4982382   2.7107925   0.52728105
  -3.5910306 ]
 [ 5.936987    6.0237923  -3.901294   -5.122726    3.5687194  -2.9142792
   1.9628466 ]
 [-1.3649368  -0.18735461 -0.6049171   2.570648   -0.259143    0.17863344
  -2.4850726 ]
 [-0.26244146  2.3950436   1.2656142  -1.9224949   2.100164    1.8372741
  -3.942083  ]
 [-1.050584    1.3406141   0.8805519   3.0829873  -0.3326287   0.16411455
  -5.621588  ]
 [-0.3388325   0.4021878  -0.40360144  1.2024801   1.0382622  -0.08057243
  -2.1980808 ]]
